In [1]:
# !pip install \
#     langchain \
#     openai \
#     google-search-results \
#     python-dotenv \
#     duckduckgo-search \
#     bittensor \
#     transformers \
#       accelerate \
#       bitsandbytes 
# scipy -q

## LLM SINGLE ACTION AGENT WITH HF AGENT

In [2]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.tools import DuckDuckGoSearchRun

#_ = load_dotenv(find_dotenv()) 

# Define which tools the agent can use to answer user queries
#search = SerpAPIWrapper(serpapi_api_key=os.environ['SERP_API_KEY'])
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [4]:
# Set up the base template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}"""

In [5]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

prompt

CustomPromptTemplate(input_variables=['input', 'intermediate_steps'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nQuestion: {input}\n{agent_scratchpad}', tools=[Tool(name='Search', description='useful for when you need to answer questions about current events', func=<bound method BaseTool.run of DuckDuckGoSearchRun()>)])

In [6]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [7]:
output_parser = CustomOutputParser()

In [8]:
import torch
from langchain.prompts import PromptTemplate
from langchain.llms.huggingface_pipeline import HuggingFacePipeline


#model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"


# Loads Zephyr-7B-beta model
# llm = HuggingFacePipeline.from_model_id(
#     model_id="HuggingFaceH4/zephyr-7b-beta",
#     task="text-generation",
#     device=0,  # replace with device_map="auto" to use the accelerate library.
#     #device_map="cuda:0",
#     pipeline_kwargs={"max_new_tokens": 1024},
#     model_kwargs={ "torch_dtype": torch.bfloat16 }
# )

# Loads SOLAR
# llm = HuggingFacePipeline.from_model_id(
#     model_id="Upstage/SOLAR-10.7B-Instruct-v1.0",
#     task="text-generation",
#     device=0,  # replace with device_map="auto" to use the accelerate library.
#     #device_map="cuda:0",
#     pipeline_kwargs={"max_new_tokens": 1024},
#     model_kwargs={ "torch_dtype": torch.float16 }
# )

# Loads Mixtral
llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    #device_map="cuda:0",
    pipeline_kwargs={"max_new_tokens": 1024},
    model_kwargs={ "torch_dtype": torch.float16, "load_in_4bit": True }
)

/home/ubuntu/pedro/envs/agent-lab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 19/19 [00:25<00:00,  1.33s/it]
Setting the `device` argument to None from 0 to avoid the error caused by attempting to move the model that was already loaded on the GPU using the Accelerate module to the same or another device.


In [9]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [11]:
from langchain.globals import set_debug

set_debug(True)
#set_verbose(True)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("How many people live in canada as of 2023?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/ubuntu/pedro/envs/agent-lab/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many people live in canada as of 2023?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How many people live in canada as of 2023?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the f

/home/ubuntu/pedro/envs/agent-lab/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:223: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference.')


[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] [14.48s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: I don't know the current population of Canada off the top of my head, I should search for it.\nAction: Search\nAction Input: Canada population 2023",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [14.48s] Exiting Chain run with output:
{
  "text": "Thought: I don't know the current population of Canada off the top of my head, I should search for it.\nAction: Search\nAction Input: Canada population 2023"
}
[tool/start] [1:chain:AgentExecutor > 4:tool:Search] Entering Tool run with input:
"Canada population 2023"
[tool/start] [1:chain:AgentExecutor > 4:tool:Search > 5:tool:duckduckgo_search] Entering Tool run with input:
"Canada population 2023"


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[tool/end] [1:chain:AgentExecutor > 4:tool:Search > 5:tool:duckduckgo_search] [1.03s] Exiting Tool run with output:
"The agency says the population growth over the first nine months of 2023 has already surpassed the total growth in any other full year, including the record set in 2022. That has been fuelled... The 2021 Canadian census had a total population count of 36,991,981 individuals, making up approximately 0.5% of the world's total population. [4] [18] A population estimate for 2023 put the total number of people in Canada at 40,097,761. [19] [20] Demographic statistics according to the World Population Review in 2022. [21] Canada's population reached 40 million on June 16, 2023, with a record-high growth rate of 2.9% since 1957, according to Statistics Canada. The increase was mostly due to net international migration, with 2% from births and deaths and 98% from non-permanent residents. October 1, 2023 1.1% (quarterly change) Source (s): Table 17-10-0009-01 . Quarterly populati

'40,097,761 people'

In [12]:
agent_executor.run("Who is the current President of Brazil?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who is the current President of Brazil?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "Who is the current President of Brazil?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final a

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[tool/end] [1:chain:AgentExecutor > 4:tool:Search > 5:tool:duckduckgo_search] s] Exiting Tool run with output:
"Brazil's new President Luiz Inacio Lula da Silva gestures as he is sworn in at the National Congress, in Brasilia, Brazil, January 1, 2023. Jacqueline Lisboa/Reuters The Senate president... President Luiz Inácio Lula da Silva took the reins of the Brazilian government on Sunday in an elaborate inauguration, complete with a motorcade, music festival and hundreds of thousands of... Joshua Kurlantzick Xi Jinping and Joe Biden Compete to Win Over Vietnam, the Region's Critical Partner Will Freeman Visualizing 2024: Trends to Watch Brazil faces a daunting set of challenges,... Luiz Inácio Lula da Silva assumed office as Brazil's new leader Sunday, just three years after being released from prison on corruption charges and 12 years after his first two terms as... Brazil's president, Luiz Inácio Lula da Silva, and his wife, Rosangela 'Janja', arrive for the Independence Day parade i

'The current President of Brazil is Luiz Inácio Lula da Silva.'

In [13]:
search.run("Current President of Brazil")

[tool/start] [1:tool:duckduckgo_search] Entering Tool run with input:
"Current President of Brazil"
[tool/end] [1:tool:duckduckgo_search] [1.06s] Exiting Tool run with output:
"Brazil's new President Luiz Inacio Lula da Silva gestures as he is sworn in at the National Congress, in Brasilia, Brazil, January 1, 2023. Jacqueline Lisboa/Reuters The Senate president... President Luiz Inácio Lula da Silva took the reins of the Brazilian government on Sunday in an elaborate inauguration, complete with a motorcade, music festival and hundreds of thousands of... Joshua Kurlantzick Xi Jinping and Joe Biden Compete to Win Over Vietnam, the Region's Critical Partner Will Freeman Visualizing 2024: Trends to Watch Brazil faces a daunting set of challenges,... Luiz Inácio Lula da Silva assumed office as Brazil's new leader Sunday, just three years after being released from prison on corruption charges and 12 years after his first two terms as... Luiz Inacio Lula da Silva has been sworn in as the new 

"Brazil's new President Luiz Inacio Lula da Silva gestures as he is sworn in at the National Congress, in Brasilia, Brazil, January 1, 2023. Jacqueline Lisboa/Reuters The Senate president... President Luiz Inácio Lula da Silva took the reins of the Brazilian government on Sunday in an elaborate inauguration, complete with a motorcade, music festival and hundreds of thousands of... Joshua Kurlantzick Xi Jinping and Joe Biden Compete to Win Over Vietnam, the Region's Critical Partner Will Freeman Visualizing 2024: Trends to Watch Brazil faces a daunting set of challenges,... Luiz Inácio Lula da Silva assumed office as Brazil's new leader Sunday, just three years after being released from prison on corruption charges and 12 years after his first two terms as... Luiz Inacio Lula da Silva has been sworn in as the new president of Brazil - the third time he has held the country's highest office. The veteran left-wing politician, known widely as Lula,..."

In [14]:
!pip show langchain

Name: langchain
Version: 0.0.351
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: /home/ubuntu/pedro/envs/agent-lab/lib/python3.8/site-packages
Requires: PyYAML, langchain-community, aiohttp, requests, async-timeout, langsmith, numpy, dataclasses-json, SQLAlchemy, pydantic, tenacity, langchain-core, jsonpatch
Required-by: 


In [15]:
!pip show transformers

Name: transformers
Version: 4.36.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/ubuntu/pedro/envs/agent-lab/lib/python3.8/site-packages
Requires: filelock, safetensors, numpy, requests, tqdm, regex, huggingface-hub, packaging, tokenizers, pyyaml
Required-by: 


In [16]:
!python --version

Python 3.8.10
